In [26]:
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid", {
 'axes.spines.bottom': True,
 'axes.spines.left': True,
 'axes.spines.right': True,
 'axes.spines.top': True
})
sns.set(font_scale=1)
#sns.set_style("darkgrid")
#sns.set_context("poster")
import sys
import os
from scipy.optimize import newton, minimize, fsolve
from scipy.interpolate import InterpolatedUnivariateSpline, krogh_interpolate, PchipInterpolator, interp1d
import numpy as np
import copy
import re
from pathlib import Path

import neutcurve
from neutcurve.colorschemes import CBMARKERS, CBPALETTE

import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import matplotlib.colors as mcolors
palette = list(mcolors.TABLEAU_COLORS.keys())
palette.extend(['salmon', 'teal','yellowgreen'])
%matplotlib inline

import random
from pyswarm import pso

In [31]:
element_prob_dict = {
    'N_natural': 0.00364,
    'N_enriched': 0.997690
}

In [41]:
def obj_iso(v1, element_prob_dict, element_list, print_sim=False):
    N_trials = 10000000
    N_enriched = int(N_trials * v1)
    N_natural = N_trials - N_enriched
    
    # For enriched:
    element_prob = [element_prob_dict['N_enriched'], element_prob_dict['N_natural']]
    ra1 = np.random.binomial(1, element_prob, (N_enriched, len(element_prob)))
    
    # For natural:
    element_prob = [element_prob_dict['N_natural'], element_prob_dict['N_natural']]
    ra2 = np.random.binomial(1, element_prob, (N_natural, len(element_prob)))
    
    ra = np.vstack((ra1, ra2))
    sim_data = np.bincount(ra.sum(1)) / len(ra) * 100
    # Adjust:
    loss = abs(obs_data[0] - sim_data[0]) + abs(obs_data[1] - sim_data[1]) + abs(obs_data[2] - sim_data[2])
    print(v1, sim_data, loss)
    if print_sim:
        return(sim_data)
    else:
        return(loss)

In [42]:
# Gln 15N1. m+0,1,2 abundance data:
obs_data = {
    0: 0.004649617339763142*100,
    1: 0.9925101283411918*100,
    2: 0.0028402543190450516
}
element_list = ['N']*2

In [43]:
# Use particle swarm to find best probability to fit the data:
def iso_fun(v1): return(obj_iso(v1, element_prob_dict, element_list))
xopt, fopt = pso(iso_fun, (0.95,), (0.9999999,), swarmsize=20, maxiter=20, minstep=1e-7, minfunc=1e-7)

[0.98813694] [ 1.40375 98.23962  0.35663] 2.303970845823819
[0.96023652] [ 4.17045 95.48076  0.34879] 7.8216908458238175
[0.99801958] [ 0.42729 99.21089  0.36182] 0.4367743137764573
[0.97159637] [ 3.04488 96.60131  0.35381] 5.580590845823823
[0.96774094] [ 3.42215 96.22595  0.3519 ] 6.331310845823809
[0.97076063] [ 3.12622 96.5226   0.35118] 5.738010845823823
[0.99831082] [ 0.39739 99.24235  0.36026] 0.43365431377644753
[0.98035514] [ 2.17894 97.4634   0.35766] 3.856410845823814
[0.96621083] [ 3.57778 96.07263  0.34959] 6.637950845823817
[0.96197972] [ 3.99389 95.65891  0.3472 ] 7.465390845823829
[0.95326749] [ 4.85802 94.79818  0.3438 ] 9.18685084582382
[0.99988217] [ 0.24083 99.39443  0.36474] 0.7294486455380889
[0.97718144] [ 2.49031 97.1544   0.35529] 4.474410845823826
[0.95299678] [ 4.88935 94.76645  0.3442 ] 9.250310845823815
[0.99371629] [ 0.85178 98.78794  0.36028] 1.2073308458238292
[0.99893419] [ 0.33559 99.30105  0.36336] 0.5399286455380925
[0.9665727] [ 3.53918 96.1081   0.

[0.9999999] [ 0.23081 99.4067   0.36249] 0.7494886455380896
[0.99977592] [ 0.25232 99.38304  0.36464] 0.706468645538083
[0.99793102] [ 0.43701 99.19865  0.36434] 0.44181431377644864
[0.9999999] [ 0.23116 99.40594  0.3629 ] 0.7487886455380901
[0.99971922] [ 0.2578  99.37819  0.36401] 0.6955086455380926
[0.99860487] [ 0.36638 99.27044  0.36318] 0.47834864553809675
[0.99798732] [ 0.42824 99.20477  0.36699] 0.447114313776453
[0.9999999] [ 0.22772 99.40927  0.36301] 0.7556686455380954
[0.99907902] [ 0.31895 99.31517  0.36588] 0.5732086455380838
[0.99840972] [ 0.38585 99.24829  0.36586] 0.4448543137764521
[0.99915475] [ 0.31351 99.32335  0.36314] 0.5840886455380938
[0.99924859] [ 0.30454 99.33223  0.36323] 0.6020286455380846
[0.9999999] [ 0.22775 99.40998  0.36227] 0.7556086455380934
[0.9999999] [ 0.22816 99.41005  0.36179] 0.7547886455381014
[0.99972286] [ 0.25535 99.37986  0.36479] 0.7004086455380826
[0.99969187] [ 0.26084 99.37522  0.36394] 0.6894286455380878
[0.99913345] [ 0.31675 99.320

[0.99756778] [ 0.47012 99.1683   0.36158] 0.4466108458238188
[0.99784858] [ 0.44464 99.19238  0.36298] 0.4390943137764493
[0.9979877] [ 0.42796 99.20666  0.36538] 0.44389431377644994
[0.99763181] [ 0.46452 99.17005  0.36543] 0.44399431377644594
[0.99776914] [ 0.45397 99.1799   0.36613] 0.44539431377644584
[0.99768778] [ 0.45841 99.17929  0.3623 ] 0.4377343137764547
[0.99785885] [ 0.43961 99.19552  0.36487] 0.44287431377644737
[0.99737593] [ 0.48946 99.14653  0.36401] 0.49015084582382246
[0.99776406] [ 0.45218 99.18868  0.35914] 0.4314143137764584
[0.99768464] [ 0.45821 99.17853  0.36326] 0.4396543137764543
[0.99777023] [ 0.44912 99.18747  0.36341] 0.43995431377644467
[0.99758196] [ 0.46776 99.16885  0.36339] 0.44551084582382894
[0.99759649] [ 0.4672  99.16957  0.36323] 0.44407084582382783
[0.99763987] [ 0.46253 99.1769   0.36057] 0.43427431377644593
[0.99764329] [ 0.46145 99.17711  0.36144] 0.43601431377645034
[0.99782858] [ 0.44685 99.19097  0.36218] 0.4374943137764421
[0.99791853] [ 

[0.99776368] [ 0.45039 99.18827  0.36134] 0.43581431377644647
[0.99799374] [ 0.42838 99.21078  0.36084] 0.4348143137764496
[0.99792347] [ 0.43497 99.19952  0.36551] 0.44415431377645676
[0.99774038] [ 0.45143 99.1866   0.36197] 0.4370743137764508
[0.9977285] [ 0.45683 99.17914  0.36403] 0.44119431377644547
[0.99765329] [ 0.46204 99.17688  0.36108] 0.4352943137764523
[0.99779476] [ 0.44828 99.18861  0.36311] 0.43935431377645223
[0.99767157] [ 0.46207 99.17457  0.36336] 0.43985431377646067
[0.99796529] [ 0.43325 99.20502  0.36173] 0.43659431377644464
[0.9976655] [ 0.46201 99.17465  0.36334] 0.43981431377644953
[0.99765213] [ 0.46136 99.17892  0.35972] 0.43257431377644423
[0.99767811] [ 0.4583  99.17839  0.36331] 0.43975431377644186
[0.9977609] [ 0.4514  99.18401  0.36459] 0.4423143137764487
[0.99770743] [ 0.45918 99.17768  0.36314] 0.4394143137764541
[0.99770784] [ 0.45872 99.1766   0.36468] 0.4424943137764417
Stopping search: maximum iterations reached --> 20


In [44]:
# Print probability:
xopt[0], fopt

(0.9976507409353106, 0.42771431377646013)

In [45]:
# Get predictions:
obj_iso(xopt[0], element_prob_dict, element_list, print_sim=True)

0.9976507409353106 [ 0.46277 99.17288  0.36435] 0.44183431377645715


array([ 0.46277, 99.17288,  0.36435])